# PikaPikaGen: Training del Modello

Questo notebook automatizza il processo di setup e avvio del training per il modello PikaPikaGen.

I passaggi eseguiti sono:
1.  Clonazione del repository GitHub pubblico.
2.  Installazione delle dipendenze necessarie tramite `uv`.
3.  Esecuzione dello script di training `main.py`.

In [ ]:
print("Installazione delle dipendenze necessarie...")

# Assicurati che uv sia installato
%pip install uv
print("✅ uv installato con successo.")

# Controlla se torch è già installato
try:
    import torch
    print(f"✅ PyTorch già installato (versione: {torch.__version__})")
    torch_installed = True
except ImportError:
    print("❌ PyTorch non trovato, sarà installato")
    torch_installed = False

# Lista delle dipendenze principali del progetto
dependencies = [
    "transformers",
    "pandas",
    "tqdm",
    "matplotlib",
    "Pillow",
    "requests",
    "ipywidgets"
]

# Aggiungi torch e torchvision solo se non sono già installati
if not torch_installed:
    dependencies.extend(["torch", "torchvision"])

print("Installazione delle dipendenze con uv...")
deps_str = " ".join(dependencies)
if torch_installed:
    !uv pip install {deps_str}
else:
    !uv pip install {deps_str} --torch-backend=auto
print("✅ Dipendenze principali installate con successo.")


In [ ]:
import os

repo_url = "https://github.com/val-2/DeepLearning"
repo_name = repo_url.split('/')[-1]

print(f"Clonazione del repository: {repo_url}")
# Clona il repository se non è già presente
if not os.path.exists(repo_name):
    !git clone {repo_url}
else:
    print(f"La directory '{repo_name}' esiste già. Aggiornamento del repository...")
    os.chdir(repo_name)
    !git fetch
    !git pull
    os.chdir('..')

# Spostati nella directory del repository
os.chdir(repo_name)
print(f"Directory di lavoro corrente: {os.getcwd()}")

In [ ]:
print("Avvio dello script di training 'main.py'...")
%run pikapikagen/main.py